In [1]:
import pandas as pd

# load data
data = pd.read_csv("data/scrapped_guttenberg_religion_paranormal.csv")
data.shape

(4467, 37)

In [2]:
# filter out data wo Summary for dataframe loader
data = data.loc[~data.Summary.isnull()]

In [3]:
from langchain_community.document_loaders import DataFrameLoader

loader = DataFrameLoader(data, page_content_column="Summary")
docs = loader.load()

In [4]:
# docs[877].page_content
# docs[877].metadata

In [5]:
# usin csv loader, metadata is useless
# from langchain_community.document_loaders.csv_loader import CSVLoader
# loader_csv = CSVLoader("data/scrapped_guttenberg_travel_geography.csv")
# docs_csv = loader_csv.load()
# docs_csv
# docs_csv[766].page_content
# docs_csv[766].metadata

In [6]:
# load embedding from azure
import os
from dotenv import load_dotenv, find_dotenv
_ = load_dotenv(find_dotenv()) # read local .env file

azure_openai_api_key = os.getenv("AZURE_OPENAI_API_KEY_4")
azure_openai_api_endpoint = os.getenv("AZURE_OPENAI_API_ENDPOINT_4")
deployment_name = os.getenv("AZURE_DEPLOYMENT_NAME_4")

from langchain.embeddings import AzureOpenAIEmbeddings 
embedding_model = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,
                                    azure_deployment='text-embedding-3-large',
                                    azure_endpoint=azure_openai_api_endpoint,
                                    openai_api_version="2023-05-15",
                                    chunk_size=500
)

/var/folders/dv/gzhyqctn53s9bh23g7tbvl940000gn/T/ipykernel_30970/4084954619.py:11: LangChainDeprecationWarning: The class `AzureOpenAIEmbeddings` was deprecated in LangChain 0.0.9 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-openai package and should be used instead. To use it run `pip install -U :class:`~langchain-openai` and import as `from :class:`~langchain_openai import AzureOpenAIEmbeddings``.
  embedding_model = AzureOpenAIEmbeddings(openai_api_key=azure_openai_api_key,


#### FAISS documentation
https://python.langchain.com/api_reference/community/vectorstores/langchain_community.vectorstores.faiss.FAISS.html#langchain_community.vectorstores.faiss.FAISS

In [8]:
# using FAISS, we keep all information
from langchain_community.vectorstores import FAISS

metadatas = [doc.metadata for doc in docs]
texts = [doc.page_content for doc in docs]
faiss_vector_store = FAISS.from_texts(texts,embedding_model,metadatas)
faiss_vector_store.save_local('data','religion_paranormal')

### IMPORTANT : use compressor when usiong the full text

USING FAISS HAS SEVERAL ADVANTAGE FOR RETRIEVAL
* max_marginal_relevance_search_with_score_by_vector()
* similarity_search_with_score_by_vector()

In [47]:
# similarity_search_with_score_by_vector(embedding: List[float], 
#                                        k: int = 4, 
#                                        filter: Callable | Dict[str, Any] | None = None, 
#                                        fetch_k: int = 20, **kwargs: Any) 
# returns → List[Tuple[Document, float]]

# similarity_search_with_score(query: str, 
#                              k: int = 4, 
#                              filter: Callable | Dict[str, Any] | None = None, 
#                              fetch_k: int = 20, 
#                              **kwargs: Any) 
# returns → List[Tuple[Document, float]]

similar = faiss_vector_store.similarity_search_with_score("Jesus is back and alive, he will smash all",
                                   k=4,
                                   fetch_k=20)

on verifie que l'embeding fait aussi les metadata

In [52]:
for sim in similar:
    print("score",sim[1])
    print("content", sim[0].page_content)
    print()

score 1.3410089
content "His Glorious Appearing: An Exposition of Matthew Twenty-Four" by James White is a religious treatise written in the late 19th century. The book delves into the biblical prophecies related to the second coming of Christ as articulated in Matthew 24, exploring themes of warning, faithfulness, and the signs that indicate the nearness of the end times. It seeks to provide a comprehensive understanding of these prophesied events for the spiritually attuned reader.  At the start of the text, White emphasizes God's historical pattern of revealing divine intentions to prophets as a precursor to judgment. He recounts biblical examples such as Noah and Lot to illustrate the concept of warnings issued before calamity, drawing a parallel to the expected return of Jesus Christ. The introductory section sets the stage for a detailed examination of Christ's prophecies concerning the end of the world, urging readers not to ignore the signs of His coming and to remain vigilant 

# Faiss vectorstore can be used as retreiver

In [ ]:
retriever = faiss_vector_store.as_retriever(
    search_type="mmr",
    search_kwargs={"k": 1, "fetch_k": 2, "lambda_mult": 0.5},
)
retriever.invoke("thud")

# load local

In [ ]:
# load_local(folder_path: str, 
#            embeddings: Embeddings, 
#            index_name: str = 'index', 
#            *, 
#            allow_dangerous_deserialization: bool = False, 
#            **kwargs: Any) 
# returns → FAISS

In [ ]:
faiss_vector_store = FAISS.load_local('data', 
                                embeddings=embedding_model
                                , index_name = 'travel_geography')